<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Note-for-Chap-4-Estimating-Point-Correspondence" data-toc-modified-id="Note-for-Chap-4-Estimating-Point-Correspondence-1">Note for Chap 4 Estimating Point Correspondence</a></span><ul class="toc-item"><li><span><a href="#From-Photometry-to-Geometry" data-toc-modified-id="From-Photometry-to-Geometry-1.1">From Photometry to Geometry</a></span><ul class="toc-item"><li><span><a href="#Non-rigid-Deformation" data-toc-modified-id="Non-rigid-Deformation-1.1.1">Non-rigid Deformation</a></span></li><li><span><a href="#Small-Deformation-versus-Wide-Baseline" data-toc-modified-id="Small-Deformation-versus-Wide-Baseline-1.1.2">Small Deformation versus Wide Baseline</a></span></li></ul></li><li><span><a href="#Small-Deformation-&amp;-Optical-Flow" data-toc-modified-id="Small-Deformation-&amp;-Optical-Flow-1.2">Small Deformation &amp; Optical Flow</a></span><ul class="toc-item"><li><span><a href="#Small-Deformation" data-toc-modified-id="Small-Deformation-1.2.1">Small Deformation</a></span></li><li><span><a href="#Optic-Flow-Estimation" data-toc-modified-id="Optic-Flow-Estimation-1.2.2">Optic Flow Estimation</a></span></li></ul></li><li><span><a href="#The-Lucas-Kanade-Method" data-toc-modified-id="The-Lucas-Kanade-Method-1.3">The Lucas-Kanade Method</a></span><ul class="toc-item"><li><span><a href="#The-Lucas-Kanade-Method" data-toc-modified-id="The-Lucas-Kanade-Method-1.3.1">The Lucas-Kanade Method</a></span></li><li><span><a href="#Estimating-Local-Displacements" data-toc-modified-id="Estimating-Local-Displacements-1.3.2">Estimating Local Displacements</a></span></li><li><span><a href="#When-can-Small-Motion-be-Estimated?" data-toc-modified-id="When-can-Small-Motion-be-Estimated?-1.3.3">When can Small Motion be Estimated?</a></span></li><li><span><a href="#A-Simple-Feature-Tracking-Algorithm" data-toc-modified-id="A-Simple-Feature-Tracking-Algorithm-1.3.4">A Simple Feature Tracking Algorithm</a></span></li></ul></li><li><span><a href="#Feature-Point-Extraction" data-toc-modified-id="Feature-Point-Extraction-1.4">Feature Point Extraction</a></span><ul class="toc-item"><li><span><a href="#Robust-Feature-Point-Extraction" data-toc-modified-id="Robust-Feature-Point-Extraction-1.4.1">Robust Feature Point Extraction</a></span></li></ul></li><li><span><a href="#Wide-Baseline-Matching" data-toc-modified-id="Wide-Baseline-Matching-1.5">Wide Baseline Matching</a></span><ul class="toc-item"><li><span><a href="#Wide-Baseline-Matching" data-toc-modified-id="Wide-Baseline-Matching-1.5.1">Wide Baseline Matching</a></span></li><li><span><a href="#Extensions-to-Larger-Baseline" data-toc-modified-id="Extensions-to-Larger-Baseline-1.5.2">Extensions to Larger Baseline</a></span></li><li><span><a href="#Normalized-Cross-Correlation" data-toc-modified-id="Normalized-Cross-Correlation-1.5.3">Normalized Cross Correlation</a></span></li><li><span><a href="#Special-Case:-Optimal-Affine-Transformation" data-toc-modified-id="Special-Case:-Optimal-Affine-Transformation-1.5.4">Special Case: Optimal Affine Transformation</a></span></li></ul></li></ul></li></ul></div>

## Note for Chap 4 Estimating Point Correspondence

- bilibili视频[lecture6](https://www.bilibili.com/video/av24149942?p=6)

- 参考书目

```
Ma, Yi, et al. An invitation to 3-d vision: from images to geometric models. Vol. 26. Springer Science & Business Media, 2012.
```

### From Photometry to Geometry

In practice, we do not actually observe points or lines, but rather **brightness or color values** at the individual pixels

主要关注光照和颜色的变化而非直接关注点与线的变化

- photometric representation ==> geometric epresentation
    - 使用特征表达点identify points with characteristic image features
    - 寻找关联性：associate these points with corresponding points

通过关键点的匹配可以帮助探知3D结构，但是，本方法是次优的：选出少量关键点，然后会丢失掉大量可能存在有用信息的特征点（this
approach is suboptimal: By **selecting a small number of feature points** from each image, we **throw away a large amount of potentially useful information** contained in each image）

- Limitations：
    - retaining all image information is computationally challenging
    - tracking of 3D objects from a moving camera in real time - even with limited processing power
    

#### Non-rigid Deformation

在一般算法中，假设物体进行刚体运动：但是现实世界中有可能发生变形（deformation）与遮挡（occlusion）

#### Small Deformation versus Wide Baseline

在点的匹配主要看两种情况——小形变（Small deformation）与（Wide baseline stereo）：

- Small deformation：one frame to the other is assumed to be (infinitesimally) small，使用光流方法解决
    - Lucas/Kanade (sparse)
    - Horn/Schunck (dense)
    one can also track the displacement of a few feature points which is typically faster （跟踪一系列点还可以较快的速度）

- Wide baseline stereo：the displacement is assumed to be large.
    - one typically selects a small number of feature points in each
of the images and develops efficient methods to find an appropriate pairing of points.（需要开发出更为高效的方法去实现实时少数点的匹配）

### Small Deformation & Optical Flow

#### Small Deformation

> 参考书目第70页

one can envision two opposite strategies: 
- **one is to choose a complex transformation that captures the changes undergone by the entire image**, or 
- **one can pick a simple transformation, and then restrict the attention to only those regions in the image whose motion can be captured, within reasonable bounds, by the chosen transformation.**

本部分主要主要关注后面的情况，假设对一静止物体的不同角度进行建模，现有图上对应点$ \mathbf{x_1}, \mathbf{x_2} $；假设两相机间的位移为刚体运动$ (R, T) $（displacement between the two camera viewpoints is a rigid-body motion.），如下所示：
$$
\mathbf{x_2} = h(\mathbf{x_1}) = \frac{1}{\lambda_2(\mathbf{X})}
(R \lambda_1(\mathbf{X}) + T)
$$

在局部则可以对于运动函数$ h $进行建模（The function h describes the transformation of the domain, or "image motion,"）：

- 平移模型Translational model：

$$ h(x) = x + b $$
其中，$ b $为偏移offset

- 仿射模型Affine model：
$$ h(x) = Ax + b $$

![Two basic types of local domain W (x) deformation. Left: translational; right:
affine](./Image/Chap4_affine.jpg)

This model is a good approximation for small
planar patches parallel to the image plane undergoing an arbitrary translation and
rotation about the optical axis, and modest rotation about an axis orthogonal to it.

特别地，2D放射模型可以如下所示：
$$ h(x) = x + u(x)\ , $$
其中
$$ u(x) = S(x)p
= \left( \begin{array}{cccccc}
x & y & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & x & y & 1
\end{array} \right)
\left( \begin{array}{cccccc}
p_1 & p_2 & p_3 & p_4 & p_5 & p_6
\end{array} \right)^{\mathsf{T}}
=
\left( \begin{array}{c}
xp_1 + yp_2 + p_3 \\ xp_4 + yp_5 + p_6
\end{array} \right)
\ . $$

#### Optic Flow Estimation

### The Lucas-Kanade Method

#### The Lucas-Kanade Method

❗ 更多待补充，标题可以考虑删除

#### Estimating Local Displacements

#### When can Small Motion be Estimated?

#### A Simple Feature Tracking Algorithm

### Feature Point Extraction

#### Robust Feature Point Extraction

### Wide Baseline Matching

#### Wide Baseline Matching

❗ 更多待补充，标题可以考虑删除

#### Extensions to Larger Baseline

#### Normalized Cross Correlation

#### Special Case: Optimal Affine Transformation

❗ 更多待补充